In [1]:
import jsonlines
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint
from tqdm import tqdm

In [2]:
client = MongoClient("localhost", 27017)

In [3]:
dbs = client.list_database_names()
dbs

['admin', 'config', 'local', 'reviews', 'roots', 'test_tutorial']

In [4]:
db = client.admin

In [18]:
serverStatusResult = db.command("serverStatus")
pprint(serverStatusResult)

{'activeIndexBuilds': {'phases': {'commit': 0,
                                  'drainSideWritesTable': 0,
                                  'drainSideWritesTableOnCommit': 0,
                                  'drainSideWritesTablePreCommit': 0,
                                  'processConstraintsViolatonTableOnCommit': 0,
                                  'scanCollection': 0,
                                  'waitForCommitQuorum': 0},
                       'total': 0},
 'asserts': {'msg': 0,
             'regular': 0,
             'rollovers': 0,
             'tripwire': 0,
             'user': 35,
             'warning': 0},
 'batchedDeletes': {'batches': 0,
                    'docs': 0,
                    'stagedSizeBytes': 0,
                    'timeMillis': 0},
 'catalogStats': {'capped': 0,
                  'clustered': 0,
                  'collections': 2,
                  'internalCollections': 3,
                  'internalViews': 0,
                  'timeseries': 0

In [70]:
# client.drop_database("roots")
client.list_database_names()

['admin', 'config', 'local', 'reviews', 'test_tutorial']

In [71]:
mongodb_filename = "/home/piktus_huggingface_co/bigscience/scisearch/data/bigscience-data-mongodb/roots_en_project_gutenberg.jsonl"
roots = client.roots  # client["roots"]
batch_size = 1024
with jsonlines.open(mongodb_filename, mode="r") as reader:
    docs_batch = []
    for mongo_doc in tqdm(reader):
        if len(docs_batch) >= batch_size:
            print("inserting batch")
            roots.roots.insert_many(docs_batch)
            docs_batch = []
        else:
            docs_batch.append(mongo_doc)
    if len(docs_batch) > 0:
        roots.roots.insert_many(docs_batch)

993it [00:07, 142.95it/s]

inserting batch


1024it [00:08, 125.17it/s]


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: roots.roots index: _id_ dup key: { _id: "bigscience-data/roots_en_project_gutenberg/0" }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 'bigscience-data/roots_en_project_gutenberg/0'}, 'op': {'_id': 'bigscience-data/roots_en_project_gutenberg/0', 'text': 'Produced by Michael Hart\n\n\n\n\n\n\n\n\n\n\nA note from Michael Hart, preparer of the 0.1 version.\n\nThis file contains a number of versions of the Magna Carta,\nsome of which were a little mangled in transit.  I am sure\nour volunteers will find and correct errors I didn\'t catch,\nand that version 0.2 - 1.0 will have significant improvements,\nas well as at least one more version in Latin.\n\nVersion 1.0 may contain a dozen different versions.\n\n\n\n\n\nThe Text of Magna Carta\n\nJOHN, by the grace of God King of England, Lord of Ireland, Duke of\nNormandy and Aquitaine, and Count 

In [50]:
cursor = roots.roots.find()
i = 0
for document in cursor:
    i += 1
    pprint(document)
    print(document["text"])
    if i > 10:
        break

{'_id': 'bigscience-data/roots_ca_enriched_conllu_ancora_for_ml_training/0',
 'language': 'en',
 'meta': None,
 'segmentations': {'para_256_16': [[0, 1289]]},
 'text': 'Entre les tasques que es duran a terme hi ha la sensibilització dels '
         "pagesos sobre l'amplitud del problema en l'àmbit agrari, perquè 'són "
         'ells mateixos els qui han de defensar la pròpia salut i la de les '
         "seves pròpies famílies'; formació dels pagesos i altres persones i "
         "institucions vinculades amb l'activitat rural, en especial dels "
         'dirigents rurals; servei de documentació per totes aquelles '
         'institucions i persones que es vulguin informar sobre els diferents '
         'aspectes de la defensa de la salut laboral; assessorament sobre la '
         'nova normativa de prevenció de riscos i la promoció adequada per al '
         'seu compliment; estudi dels principals problemes que es produeixen '
         "en les condicions de treball dels pagesos; imp

In [10]:
roots = client.roots  # client["roots"]
c = roots.roots.find({"_id": "bigscience-data/roots_en_no_code_stackexchange/1495630"})

In [11]:
for doc in c:
    pprint(doc)

{'_id': 'bigscience-data/roots_en_no_code_stackexchange/1495630',
 'language': 'en',
 'meta': {'file': 'bitcoin.stackexchange_0000054267.txt'},
 'segmentations': {'para_256_16': [[0, 3980],
                                   [3694, 8701],
                                   [8284, 13464],
                                   [13109, 18211],
                                   [17945, 22631],
                                   [22316, 25658],
                                   [25569, 28465],
                                   [28067, 33205],
                                   [32874, 38070],
                                   [37625, 43110],
                                   [42684, 47713],
                                   [47409, 47978]]},
 'text': 'Q:\n'
         '\n'
         'Bitcoin-Core Developers sharing e-mail address?\n'
         '\n'
         "I've been looking into the Github commit history of Bitcoin. I have "
         "no clue how git works, but interestingly, I'm seeing a 

In [ ]:
db.command({"planCacheClear": "collection_name"})